In [1]:
import pickle
import numpy as np
import os
import preloaded_vgg22
import custom_vgg22
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Input, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.vgg19 import VGG19

In [2]:
# function provided by the dataset owner for unpickling the data
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
# loading training dataset
data = np.concatenate((np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_1')[b'data'], dtype='int16'),
                       np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_2')[b'data'], dtype='int16'),
                       np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_3')[b'data'], dtype='int16'),
                       np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_4')[b'data'], dtype='int16'),
                       np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_5')[b'data'], dtype='int16')))
# loading labels for the training data
labels = np.concatenate((np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_1')[b'labels'], dtype='int8'),
                         np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_2')[b'labels'], dtype='int8'),
                         np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_3')[b'labels'], dtype='int8'),
                         np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_4')[b'labels'], dtype='int8'),
                         np.array(unpickle('D:/CIFAR_10_stuff/data/data_batch_5')[b'labels'], dtype='int8')))

In [4]:
IMG_SIZE = (32, 32)  # image size 32x32

In [5]:
data = data.reshape((50000, 1024, 3), order='F').reshape((50000, 32, 32, 3))  # data reshaping into 32x32x3

In [6]:
# defining a data augmentation function
data_augmentation = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                       shear_range=0.2, zoom_range=0.2)

In [7]:
data_flipped = data[:, :, ::-1, :]  # flipping the images horizontally

In [8]:
# running data augmentation on original training data
data_augment = NumpyArrayIterator(data, labels, image_data_generator=data_augmentation, shuffle=False)

In [9]:
# running data augmentation on flipped training data
data_flipped_augment = NumpyArrayIterator(data_flipped, labels, image_data_generator=data_augmentation, shuffle=False)

In [10]:
# combining original data, flipped data, and augmented data to engineer entire training dataset
combined_data = np.concatenate((data_augment.x, data_flipped_augment.x, data, data_flipped), axis=0)
# doing same thing for the labels
combined_labels = np.concatenate((data_augment.y, data_flipped_augment.y, labels, labels), axis=0)

In [11]:
# storing combined training data as a Numpy Array Iterator
training_data = NumpyArrayIterator(combined_data, combined_labels, image_data_generator=ImageDataGenerator(), shuffle=True)

In [13]:
# loading validation and testing dataset
test_val_data = np.array(unpickle('D:/CIFAR_10_stuff/data/test_batch')[b'data'], dtype='int16')
test_val_labels = np.array(unpickle('D:/CIFAR_10_stuff/data/test_batch')[b'labels'], dtype='int8')

In [14]:
# splitting to get separated validation and testing datasets
X_val, X_test, y_val, y_test = train_test_split(test_val_data, test_val_labels, test_size=0.5,
                                                stratify=test_val_labels, random_state=42)

In [15]:
X_val = X_val.reshape((5000, 1024, 3), order='F').reshape((5000, 32, 32, 3))  # data reshaping into 32x32x3
X_test = X_test.reshape((5000, 1024, 3), order='F').reshape((5000, 32, 32, 3))  # data reshaping into 32x32x3

In [16]:
# storing validation and testing datasets as Numpy Array Iterators
validation_data = NumpyArrayIterator(X_val, y_val, image_data_generator=ImageDataGenerator(), shuffle=True)
test_data = NumpyArrayIterator(X_test, y_test, image_data_generator=ImageDataGenerator(), shuffle=True)

### DEFINING NECESSARY CALLBACK FUNCTIONS

In [17]:
# Early Stopping Callback
def early_stopping(patience=3):
    return EarlyStopping(patience=patience)

# Model Checkpoint Callback
def checkpoint_callback(model_name='model'):
    return ModelCheckpoint(
        filepath='D:/CIFAR_10_stuff/weights/'+model_name+'_epoch_{epoch:02d}.h5',  # saving the epoch number in the filename
        save_weights_only=True
    )

# Learning Rate Scheduler Callback
def lr_scheduler(schedule_by=0.1, schedule_at=4):
    def schedule(epoch, lr):
        if epoch % schedule_at == 0:
            return lr * schedule_by
        else:
            return lr
    return LearningRateScheduler(schedule=schedule)

# Learning Rate Reduction on Plateau Callback
def lr_red_plateau(factor=0.5, patience=5):
    return ReduceLROnPlateau(factor=factor, patience=patience, min_lr=1e-8)

### PRELOADED VGGNet MODEL WITHOUT AND WITH FINE-TUNING

##### NON-TRAINABLE VGGNet BASE MODEL

In [18]:
model = preloaded_vgg22.cifar10_VGG22()

In [19]:
base_learning_rate = 10**(-4)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [20]:
epoch_steps = int(training_data.x.shape[0] / 64)

In [21]:
model.save_weights('D:/CIFAR_10_stuff/weights/preload_VGG_initial_weights.h5')

In [87]:
initial_epochs = 8

history = model.fit(training_data, validation_data=validation_data, epochs=initial_epochs, steps_per_epoch=epoch_steps,
                    callbacks=[early_stopping(patience=4), lr_red_plateau(factor=0.5, patience=2)])

Epoch 1/8
3125/3125 [==============================] - 770s 246ms/step - loss: 1.1346 - accuracy: 0.6071 - val_loss: 1.0021 - val_accuracy: 0.6552 - lr: 1.0000e-04
Epoch 2/8
3125/3125 [==============================] - 1189s 380ms/step - loss: 1.0818 - accuracy: 0.6219 - val_loss: 0.9810 - val_accuracy: 0.6618 - lr: 1.0000e-04
Epoch 3/8
3125/3125 [==============================] - 885s 283ms/step - loss: 1.0491 - accuracy: 0.6299 - val_loss: 0.9708 - val_accuracy: 0.6722 - lr: 1.0000e-04
Epoch 4/8
3125/3125 [==============================] - 774s 248ms/step - loss: 1.0251 - accuracy: 0.6392 - val_loss: 0.9666 - val_accuracy: 0.6724 - lr: 1.0000e-04
Epoch 5/8
3125/3125 [==============================] - 847s 271ms/step - loss: 1.0107 - accuracy: 0.6435 - val_loss: 0.9553 - val_accuracy: 0.6762 - lr: 1.0000e-04
Epoch 6/8
3125/3125 [==============================] - 842s 269ms/step - loss: 0.9919 - accuracy: 0.6500 - val_loss: 0.9463 - val_accuracy: 0.6764 - lr: 1.0000e-04
Epoch 7/8
3125/

In [89]:
preds = model.evaluate(test_data)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

157/157 [==============================] - 33s 201ms/step - loss: 0.9335 - accuracy: 0.6826
Loss = 0.933518648147583
Test Accuracy = 0.6826000213623047


In [90]:
model.save_weights('D:/CIFAR_10_stuff/weights/preload_VGG_weights_without_base_model_finetuning.h5')

##### FULLY TRAINABLE VGG19 BASE MODEL

In [143]:
base_model = model.layers[3]
base_model.trainable = True

# recompiling the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.5 * base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [113]:
fin_epochs = history.epoch[-1] + 1

history_fine = model.fit(training_data,
                         epochs=int(1.5*fin_epochs),
                         initial_epoch=fin_epochs,
                         validation_data=validation_data,
                         steps_per_epoch=epoch_steps,
                         callbacks=[early_stopping(patience=3),
                                    lr_red_plateau(patience=2),
                                    checkpoint_callback(model_name='preloaded_VGG')])

Epoch 9/12
3125/3125 [==============================] - 3231s 1s/step - loss: 0.0746 - accuracy: 0.9772 - val_loss: 0.3863 - val_accuracy: 0.8876 - lr: 1.0000e-06
Epoch 10/12
 532/3125 [====>.........................] - ETA: 44:13 - loss: 0.0634 - accuracy: 0.9808

KeyboardInterrupt: 

In [144]:
preds = model.evaluate(test_data)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

157/157 [==============================] - 216s 209ms/step - loss: 0.3684 - accuracy: 0.8840
Loss = 0.3684088885784149
Test Accuracy = 0.8840000033378601


### CUSTOM VGGNet MODEL WITHOUT WEIGHT INITIALIZATION

In [22]:
model2 = custom_vgg22.cifar10_custom_VGG22()

In [23]:
custom_learning_rate = 10**(-4)
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = custom_learning_rate),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])

In [24]:
model2.save_weights('D:/CIFAR_10_stuff/weights/custom_VGG_initial_weights.h5')

In [139]:
history_custom = model2.fit(training_data,
                            epochs=8,
                            validation_data=validation_data,
                            steps_per_epoch=epoch_steps,
                            callbacks=[lr_red_plateau(patience=2),
                                       checkpoint_callback(model_name='custom_VGG')])

Epoch 1/8
3125/3125 [==============================] - 3376s 1s/step - loss: 0.3306 - accuracy: 0.9080 - val_loss: 0.7224 - val_accuracy: 0.8170 - lr: 7.5000e-06
Epoch 2/8
3125/3125 [==============================] - 3315s 1s/step - loss: 0.3119 - accuracy: 0.9150 - val_loss: 0.7311 - val_accuracy: 0.8152 - lr: 7.5000e-06
Epoch 3/8
3125/3125 [==============================] - 3427s 1s/step - loss: 0.2857 - accuracy: 0.9230 - val_loss: 0.7548 - val_accuracy: 0.8154 - lr: 3.7500e-06
Epoch 4/8
 556/3125 [====>.........................] - ETA: 44:51 - loss: 0.2781 - accuracy: 0.9270

KeyboardInterrupt: 

In [140]:
preds = model2.evaluate(test_data)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

157/157 [==============================] - 42s 258ms/step - loss: 0.7842 - accuracy: 0.8104
Loss = 0.7841576933860779
Test Accuracy = 0.8104000091552734
